In [4]:
import cv2
import numpy as np

YOLO_CONFIG = "yolov3.cfg"
YOLO_WEIGHTS = "yolov3.weights"
YOLO_NAMES = "coco.names"

CONFIDENCE_THRESHOLD = 0.5
NMS_THRESHOLD = 0.4


In [6]:
with open(YOLO_NAMES, "r") as f:  # YOLO_NAMES is name of the file, should store the class names
    classes = [line.strip() for line in f.readlines()]

net = cv2.dnn.readNetFromDarknet(YOLO_CONFIG, YOLO_WEIGHTS)

net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

print("Classes loaded:", classes)
print("Output layers:", output_layers)


FileNotFoundError: [Errno 2] No such file or directory: 'coco.names'

In [10]:
def detect_objects(frame):
  
    height, width = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(frame, 
                                 scalefactor=1/255.0, 
                                 size=(416, 416),
                                 swapRB=True, 
                                 crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(output_layers)
    
    boxes = []
    confidences = []
    class_ids = []
    
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]       
            class_id = np.argmax(scores) 
            confidence = scores[class_id]
            
            if confidence > CONFIDENCE_THRESHOLD:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    
    for i in indices:
        i = i[0]
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        conf = confidences[i]
        
        color = (0, 255, 0) 
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        text = f"{label} {conf:.2f}"
        cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return frame, (boxes, confidences, class_ids)
